In [96]:
# Render our plots inline
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

matplotlib.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 5)

train = pd.read_csv('/Users/Kseniya/study/4_term/ml/train.csv')
print train.shape
test= pd.read_csv('/Users/Kseniya/study/4_term/ml/test.csv')
print test.shape

(76020, 371)
(75818, 370)


In [97]:
def duplicate_columns(frame):
    groups = frame.columns.to_series().groupby(frame.dtypes).groups
    dups = []
    for t, v in groups.items():
        dcols = frame[v].to_dict(orient="list")

        vs = dcols.values()
        ks = dcols.keys()
        lvs = len(vs)

        for i in range(lvs):
            for j in range(i+1,lvs):
                if vs[i] == vs[j]: 
                    dups.append(ks[i])
                    break

    return dups   


def data_preprocess(data):
    # Drop constants columns
    data = data.loc[:, data.apply(pd.Series.nunique) != 1]
    
    # Drop columns duplicate
    dups = duplicate_columns(data)
    data = data.drop(dups, axis=1)
    
    # Drop row duplicates 
    data.drop_duplicates(keep='first', inplace = True)
    
    data = pd.get_dummies(data)
    
    # Findout NaN values (no such values)
    # train_data.isnull().sum().sum()
    #empty_arr = pd.isnull(data).any(1).nonzero()[0]
    #print empty_arr
    return data

def leave_same_columns(data, test):
    modified_test = pd.DataFrame()
    for col in data:
        if col in test.columns:
            modified_test[col] = test[col]
    return modified_test

print 'Before processing: '
print train.shape
print test.shape

train = data_preprocess(train)

# Drop Target column 
target = train['TARGET']
train.drop('TARGET', axis=1, inplace=True)
test = leave_same_columns(train, test)

print 'After processing: '
print train.shape
print test.shape

train_orig = train.copy()
test_orig = test.copy()

Before processing: 
(76020, 371)
(75818, 370)
After processing: 
(76020, 307)
(75818, 307)


In [98]:
%%time
import numpy as np
# categories encoding
def dummy_columns(data, category):
    for c in category:
        dummies = pd.get_dummies(data[c], prefix="{}_xxx_".format(c))
        data = pd.concat([data, dummies], axis=1)
    print data.shape
    data.drop(category, axis=1, inplace=True)
    return data

def categories_encode(data, test, n):
    # examine if there are categorical columns
    res = data.dtypes == 'object'
    print 'Train data types cnt : '
    print res.value_counts()
    res = test.dtypes == 'object'
    print 'Test data types cnt : '
    print res.value_counts()
    
    # no str data types found
    # find value sets of N elements
    category = []
    for col in data.loc[:, data.dtypes == np.int64]:
        if (data[col].value_counts().count() <= n and test[col].value_counts().count() <= n):
            category.append(col)
            
    if len(category) == 0:
        print "No category columns for n = {}".format(n)
        return None
    
    return category




CPU times: user 62 µs, sys: 248 µs, total: 310 µs
Wall time: 181 µs


In [61]:
train = train_orig.copy()
test = test_orig.copy()

In [99]:
# categories encoding
category_train = None
category_test = None

category = categories_encode(train, test, 10)
if category is not None:
    for col in category:
        dummies = pd.get_dummies(train[col], prefix="{}_xxx_".format(col))
        category_train = pd.concat([category_train, dummies], axis=1)
        
        dummies = pd.get_dummies(test[col], prefix="{}_xxx_".format(col))
        category_test = pd.concat([category_test, dummies], axis=1)
        
        train.drop(col, axis=1, inplace=True)
        test.drop(col, axis=1, inplace=True)
        
    #train = dummy_columns(train, category)
    #test = dummy_columns(test, category)

print 'After categorisation:\n Data:\n'
print train.shape
print 'Test:\n'
print test.shape
print 'Category train:\n'
print category_train.shape
print 'Category test:\n'
print category_test.shape

Train data types cnt : 
False    307
dtype: int64
Test data types cnt : 
False    307
dtype: int64
After categorisation:
 Data:

(76020, 157)
Test:

(75818, 157)
Category train:

(76020, 516)
Category test:

(75818, 503)


In [100]:
def modify_test_colomns(data, test_from):
    test = pd.DataFrame()
    index = -1
    for col in data.columns:
        index += 1
        if col not in test_from.columns:
            test[col] = pd.Series([0 for x in range(test_from.shape[0])])
        else:
            test[col] = test_from[col]
            
    #print test.shape
    
    for col in data.columns:
        if data.columns.get_loc(col) != test.columns.get_loc(col):
            print "index missmatch " + col
    #print test.shape
    return test

category_test = modify_test_colomns(category_train, category_test)

print category_train.shape
print category_test.shape


(76020, 516)
(75818, 516)


In [101]:
train_orig_1 = train.copy()
test_orig_1 = test.copy()

In [102]:
# feature selection
from sklearn.feature_selection import SelectPercentile, f_classif
# Removing features with low variance
def get_best_features(X, y, perc):
    selectK = SelectPercentile(f_classif, perc)
    selectK.fit(X[:y.shape[0]], y)
    return pd.DataFrame(selectK.transform(X))

train = train_orig_1.copy()
test = test_orig_1.copy()

print 'Before selection best features:\n Data:\n'
print train.shape
print 'Test:\n'
print test.shape

train = get_best_features(train, target, 90)
test = modify_test_colomns(train, test)

print 'After selection best features:\n Data:\n'
print train.shape
print 'Test:\n'
print test.shape

Before selection best features:
 Data:

(76020, 157)
Test:

(75818, 157)
After selection best features:
 Data:

(76020, 141)
Test:

(75818, 141)


In [68]:
train_orig_2 = train.copy()
test_orig_2 = test.copy()

In [70]:
# estimate linear models for category features only
from sklearn import cross_validation
from sklearn.linear_model import BayesianRidge, LinearRegression,LogisticRegression
classes = [
    {'type': "liner regression", 'classifier': LinearRegression, 'params': {
                                                                    'copy_X': True,
                                                                    'normalize': True,
                                                                    'n_jobs': 4}},
    {'type': "logistic regression", 'classifier': LogisticRegression, 'params': {
                                                                        'solver': 'sag',
                                                                        'multi_class': 'ovr',
                                                                        'n_jobs': 4}},
   {'type': "bayes ridge", 'classifier': BayesianRidge, 'params': {}},
    # {'type': "", 'classifier': , 'params': {}},
]

for c in classes:
    params = dict(c.get('params', {}))
    classifier = c['classifier']
    classifier = classifier(**params)
    res = cross_validation.cross_val_score(classifier, category_train, target, n_jobs=4, scoring='roc_auc').mean()
    print "{}: {}\n".format(c['type'], res)
    


liner regression: 0.708772132648



/Users/Kseniya/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/Kseniya/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/Kseniya/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


logistic regression: 0.728865553806

bayes ridge: 0.726836607321



In [87]:
# find parametrs for ligistic regression on category train
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

log_train = int(0.7 * category_train.shape[0])

dtrain = category_train[:int(log_train)]
dtarget = target[:int(log_train)]

dtest = category_train[log_train:]
dtest_target = target[log_train:]

def objective(params):
    log_clf = LogisticRegression(dual = False, penalty='l2', random_state=13, multi_class='ovr', verbose=1, 
                                n_jobs=4, solver='sag', C=params['C'], tol=params['tol'])


    log_clf.fit(dtrain, dtarget)
    dpred = log_clf.predict_proba(dtest)[:,1]
    
    auc = roc_auc_score(dtest_target, dpred)
    print "auc {} params {}".format(auc, params)
    return {'loss':1-auc, 'status': STATUS_OK }
    
params = {
    'C': hp.uniform('C', 0.2, 5),
    'tol': hp.uniform('tol', 0, 100),
}    
iteration_number = 100

trials = Trials()
best_res = fmin(fn=objective,
                space=params,
                algo=tpe.suggest,
                max_evals=iteration_number,
                trials=trials)
print "BEST "
print best_res




convergence after 1 epochs took 1 seconds
auc 0.727091568292 params {'C': 1.5734688077618213, 'tol': 69.64691855978616}


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.2s finished


convergence after 1 epochs took 0 seconds
auc 0.727020480314 params {'C': 3.7782631274509426, 'tol': 10.606490595473272}


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.2s finished


convergence after 1 epochs took 0 seconds
auc 0.727010953803 params {'C': 0.4761473368068307, 'tol': 50.672601149956996}


...

[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.2s finished


convergence after 1 epochs took 0 seconds
auc 0.727021779384 params {'C': 4.017901737520956, 'tol': 9.427472698903635}


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.2s finished


convergence after 1 epochs took 0 seconds
auc 0.727024810546 params {'C': 4.257733439994073, 'tol': 62.246658241353764}
BEST 
{'C': 3.9516364538326187, 'tol': 0.060459326089611576}


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.2s finished


In [88]:
# auc 0.745438473295 params {'C': 3.9516364538326187, 'tol': 0.060459326089611576}
category_model = LogisticRegression(dual = False, penalty='l2', random_state=13, multi_class='ovr', verbose=1, 
                                    n_jobs=4, solver='sag', C=3.9516364538326187, tol=0.060459326089611576)
category_model.fit(dtrain, dtarget)
dpred = category_model.predict_proba(dtest)[:,1]
print dpred

convergence after 10 epochs took 2 seconds


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    2.2s finished


[ 0.01835048  0.0436105   0.00032668 ...,  0.02184358  0.01896538
  0.08473333]


In [106]:
# test classifiers on all train data 
import time

from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import svm
from sklearn.linear_model import BayesianRidge, LinearRegression, LogisticRegression

classes = [
    {'type': "random_forest", 'classifier': RandomForestClassifier, 'params': {'n_estimators': 100, 'n_jobs': 4}},
    {'type': "liner regression", 'classifier': LinearRegression, 'params': {
                                                                    'copy_X': True,
                                                                    'normalize': True,
                                                                    'n_jobs': 4}},
    {'type': "logistic regression", 'classifier': LogisticRegression, 'params': {
                                                                        'solver': 'sag',
                                                                        'multi_class': 'ovr',
                                                                        'n_jobs': 4}},
   {'type': "ada boost", 'classifier': AdaBoostClassifier, 'params': {'n_estimators': 15}},
    # {'type': "", 'classifier': , 'params': {}},
]

for c in classes:
    params = dict(c.get('params', {}))
    classifier = c['classifier']
    classifier = classifier(**params)
    res = cross_validation.cross_val_score(classifier, train, target, n_jobs=4, scoring='roc_auc').mean()
    print "{}: {}\n".format(c['type'], res)
    


random_forest: 0.71659756633

liner regression: 0.777619229636



/Users/Kseniya/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/Kseniya/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/Kseniya/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


logistic regression: 0.593684156614

ada boost: 0.820528181998



In [107]:
for col in train.columns:
    if train.columns.get_loc(col) != test.columns.get_loc(col):
        print "index missmatch " + col
print "done"

done


In [104]:
# test xgboost on train data without category data
import xgboost
from sklearn.metrics import roc_auc_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

xgb_train = int(0.7 * train.shape[0])

dtrain = train[:int(xgb_train/2)]
dtarget = target[:int(xgb_train/2)]

dtrain_test = train[int(xgb_train/2):xgb_train]
dtrain_test_target = target[int(xgb_train/2):xgb_train]

dtest = train[xgb_train:]
dtest_target = target[xgb_train:]

def objective(params):
    xgb_clf = xgb.XGBClassifier(missing=np.nan, nthread=4, seed=13, 
                                 n_estimators=int(params['n_estimators']),
                                 base_score=params['base_score'],
                                 max_depth=int(params['max_depth']),
                                 max_delta_step=int(params['max_delta_step']),
                                 min_child_weight=params['min_child_weight'],
                                 learning_rate=params['learning_rate'],
                                 subsample=params['subsample'],
                                 scale_pos_weight=params['scale_pos_weight'],
                                 colsample_bytree=params['colsample_bytree'])

    xgb_clf.fit(dtrain, dtarget, eval_metric="auc", eval_set=[(dtrain_test, dtrain_test_target)], early_stopping_rounds=30)
    dpred = xgb_clf.predict_proba(dtest)[:,1]
    
    auc = roc_auc_score(dtest_target, dpred)
    print "auc {} params {}".format(auc, params)
    return {'loss':1-auc, 'status': STATUS_OK }
    
params = {
    'max_depth': hp.quniform("max_depth", 2, 6, 1),
    'max_delta_step': hp.quniform('max_delta_step', 1, 10, 1),
    'base_score': hp.uniform('base_score', 0.4, 1),
    'n_estimators': hp.quniform('n_estimators', 1000, 40000, 1),
    'min_child_weight': hp.quniform('min_child', 1, 15, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.03),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0.2, 1.0)
}    
iteration_number = 50

trials = Trials()
best_res = fmin(fn=objective,
                space=params,
                algo=tpe.suggest,
                max_evals=iteration_number,
                trials=trials)
print best_res




Will train until validation_0 error hasn't decreased in 30 rounds.
[0]	validation_0-auc:0.710925
[1]	validation_0-auc:0.710804
[2]	validation_0-auc:0.710804
[3]	validation_0-auc:0.710804
[4]	validation_0-auc:0.760027
[5]	validation_0-auc:0.762722
[6]	validation_0-auc:0.763944
[7]	validation_0-auc:0.780960
[8]	validation_0-auc:0.780025
[9]	validation_0-auc:0.780142
[10]	validation_0-auc:0.780086
[11]	validation_0-auc:0.782400
[12]	validation_0-auc:0.782345
[13]	validation_0-auc:0.780068
[14]	validation_0-auc:0.780019
[15]	validation_0-auc:0.780170
[16]	validation_0-auc:0.786236
[17]	validation_0-auc:0.790499
[18]	validation_0-auc:0.796524
[19]	validation_0-auc:0.796464
[20]	validation_0-auc:0.809441
[21]	validation_0-auc:0.813787
[22]	validation_0-auc:0.813868
[23]	validation_0-auc:0.808868
[24]	validation_0-auc:0.808815
[25]	validation_0-auc:0.808961
[26]	validation_0-auc:0.810352
[27]	validation_0-auc:0.810684
[28]	validation_0-auc:0.811150
[29]	validation_0-auc:0.811120
[30]	validati

auc 0.841002270726 params {'colsample_bytree': 0.8739318954339452, 'scale_pos_weight': 0.4289114679603036, 'learning_rate': 0.029615283967692307, 'max_delta_step': 5.0, 'base_score': 0.6885591408906166, 'n_estimators': 9847.0, 'subsample': 0.8785876742391446, 'min_child_weight': 9.0, 'max_depth': 5.0}


Will train until validation_0 error hasn't decreased in 30 rounds.
[0]	validation_0-auc:0.759656
[1]	validation_0-auc:0.783988
[2]	validation_0-auc:0.799326
[3]	validation_0-auc:0.790437
[4]	validation_0-auc:0.805313
[5]	validation_0-auc:0.796000
[6]	validation_0-auc:0.809592
[7]	validation_0-auc:0.810758
[8]	validation_0-auc:0.814087
[9]	validation_0-auc:0.816589
[10]	validation_0-auc:0.816379
[11]	validation_0-auc:0.815186
[12]	validation_0-auc:0.810310
[13]	validation_0-auc:0.811100
[14]	validation_0-auc:0.809947
[15]	validation_0-auc:0.811472
[16]	validation_0-auc:0.815408
[17]	validation_0-auc:0.813356
[18]	validation_0-auc:0.815632
[19]	validation_0-auc:0.818082
[20]	validation_0-auc:0.818709
[21]	validation_0-auc:0.819466
[22]	validation_0-auc:0.819784
[23]	validation_0-auc:0.821356
[24]	validation_0-auc:0.821772
[25]	validation_0-auc:0.821773
[26]	validation_0-auc:0.820656
[27]	validation_0-auc:0.819876
[28]	validation_0-auc:0.820964
[29]	validation_0-auc:0.821499
[30]	validati

auc 0.841122603066 params {'colsample_bytree': 0.7581900043129112, 'scale_pos_weight': 0.7963771879084904, 'learning_rate': 0.023359248188161465, 'max_delta_step': 3.0, 'base_score': 0.5689437364024854, 'n_estimators': 23320.0, 'subsample': 0.6424259623818931, 'min_child_weight': 7.0, 'max_depth': 4.0}


Will train until validation_0 error hasn't decreased in 30 rounds.
[0]	validation_0-auc:0.500000
[1]	validation_0-auc:0.500000
[2]	validation_0-auc:0.500000
[3]	validation_0-auc:0.500000
[4]	validation_0-auc:0.500000
[5]	validation_0-auc:0.500000
[6]	validation_0-auc:0.500000
[7]	validation_0-auc:0.500000
[8]	validation_0-auc:0.500000
[9]	validation_0-auc:0.500000
[10]	validation_0-auc:0.500000
[11]	validation_0-auc:0.500000
[12]	validation_0-auc:0.500000
[13]	validation_0-auc:0.710333
[14]	validation_0-auc:0.710333
[15]	validation_0-auc:0.710333
[16]	validation_0-auc:0.710333
[17]	validation_0-auc:0.710333
[18]	validation_0-auc:0.711043
[19]	validation_0-auc:0.711042
[20]	validation_0-auc:0.711042
[21]	validation_0-auc:0.711563
[22]	validation_0-auc:0.711564
[23]	validation_0-auc:0.760938
[24]	validation_0-auc:0.762086
[25]	validation_0-auc:0.770642
[26]	validation_0-auc:0.757085
[27]	validation_0-auc:0.758386
[28]	validation_0-auc:0.757085
[29]	validation_0-auc:0.759038
[30]	validati

...

auc 0.830903640065 params {'colsample_bytree': 0.8160651622834605, 'scale_pos_weight': 0.3194680632155796, 'learning_rate': 0.02158798199520957, 'max_delta_step': 7.0, 'base_score': 0.8262470471168736, 'n_estimators': 2278.0, 'subsample': 0.7952934916304413, 'min_child_weight': 5.0, 'max_depth': 3.0}


Will train until validation_0 error hasn't decreased in 30 rounds.
[0]	validation_0-auc:0.500000
[1]	validation_0-auc:0.500000
[2]	validation_0-auc:0.500000
[3]	validation_0-auc:0.500000
[4]	validation_0-auc:0.500000
[5]	validation_0-auc:0.500000
[6]	validation_0-auc:0.500000
[7]	validation_0-auc:0.500000
[8]	validation_0-auc:0.500000
[9]	validation_0-auc:0.500000
[10]	validation_0-auc:0.500000
[11]	validation_0-auc:0.500000
[12]	validation_0-auc:0.500000
[13]	validation_0-auc:0.500000
[14]	validation_0-auc:0.500000
[15]	validation_0-auc:0.500000
[16]	validation_0-auc:0.500000
[17]	validation_0-auc:0.500000
[18]	validation_0-auc:0.500000
[19]	validation_0-auc:0.678024
[20]	validation_0-auc:0.710344
[21]	validation_0-auc:0.710344
[22]	validation_0-auc:0.710344
[23]	validation_0-auc:0.760944
[24]	validation_0-auc:0.760944
[25]	validation_0-auc:0.760944
[26]	validation_0-auc:0.760944
[27]	validation_0-auc:0.760944
[28]	validation_0-auc:0.760944
[29]	validation_0-auc:0.760944
[30]	validati

auc 0.841272429099 params {'colsample_bytree': 0.6699839087191768, 'scale_pos_weight': 0.45038079775921824, 'learning_rate': 0.010731457848557755, 'max_delta_step': 6.0, 'base_score': 0.9281751280068482, 'n_estimators': 5839.0, 'subsample': 0.7214679401811748, 'min_child_weight': 6.0, 'max_depth': 5.0}


Will train until validation_0 error hasn't decreased in 30 rounds.
[0]	validation_0-auc:0.710925
[1]	validation_0-auc:0.773915
[2]	validation_0-auc:0.773915
[3]	validation_0-auc:0.759795
[4]	validation_0-auc:0.762509
[5]	validation_0-auc:0.777442
[6]	validation_0-auc:0.779798
[7]	validation_0-auc:0.781874
[8]	validation_0-auc:0.781229
[9]	validation_0-auc:0.785056
[10]	validation_0-auc:0.786142
[11]	validation_0-auc:0.784548
[12]	validation_0-auc:0.784865
[13]	validation_0-auc:0.785908
[14]	validation_0-auc:0.779184
[15]	validation_0-auc:0.780084
[16]	validation_0-auc:0.801126
[17]	validation_0-auc:0.798882
[18]	validation_0-auc:0.799202
[19]	validation_0-auc:0.797024
[20]	validation_0-auc:0.797774
[21]	validation_0-auc:0.805068
[22]	validation_0-auc:0.801713
[23]	validation_0-auc:0.801645
[24]	validation_0-auc:0.799726
[25]	validation_0-auc:0.804806
[26]	validation_0-auc:0.803492
[27]	validation_0-auc:0.802391
[28]	validation_0-auc:0.798440
[29]	validation_0-auc:0.803496
[30]	validati

auc 0.840439364601 params {'colsample_bytree': 0.710955267817827, 'scale_pos_weight': 0.5297986988018918, 'learning_rate': 0.020020473914413705, 'max_delta_step': 9.0, 'base_score': 0.7637105054651051, 'n_estimators': 10199.0, 'subsample': 0.7817688404200562, 'min_child_weight': 3.0, 'max_depth': 4.0}
{'colsample_bytree': 0.6199986650131059, 'scale_pos_weight': 0.30251525762720033, 'learning_rate': 0.021600903484051248, 'max_delta_step': 4.0, 'base_score': 0.6592170905114092, 'n_estimators': 4409.0, 'subsample': 0.6420688508870701, 'min_child': 3.0, 'max_depth': 5.0}


In [ ]:
#best params 
auc 0.842189572272 params {'colsample_bytree': 0.6199986650131059, 'scale_pos_weight': 0.30251525762720033, 'learning_rate': 0.021600903484051248, 'max_delta_step': 4.0, 'base_score': 0.6592170905114092, 'n_estimators': 4409.0, 'subsample': 0.6420688508870701, 'min_child_weight': 3.0, 'max_depth': 5.0}

In [106]:
# try to blend two classifiers: logistic regression for category data and xgboost for (train - category) data 
from sklearn.linear_model import LogisticRegression
import xgboost
from sklearn.metrics import roc_auc_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

model_train = int(0.7 * category_train.shape[0])
# for xgb
dtrain = train[:int(model_train/2)]
dtarget = target[:int(model_train/2)]

dtrain_test = train[int(model_train/2):model_train]
dtrain_test_target = target[int(model_train/2):model_train]

# for log regression
dcat_train = category_train[:int(model_train)]
dcat_target = target[:int(model_train)]

dtest = train[model_train:]
dcat_test = category_train[model_train:]
dtest_target = target[model_train:]

xgb_clf = xgb.XGBClassifier(missing=np.nan, nthread=4, seed=13, 
                            scale_pos_weight =  0.30251525762720033,
                            learning_rate = 0.021600903484051248,
                            max_delta_step = 9.0,
                            base_score = 0.6592170905114092,
                            n_estimators = 4409,
                            subsample = 0.6420688508870701,
                            min_child_weight = 3.0,
                            max_depth = 5.0,
                            colsample_bytree=0.6199986650131059)
xgb_clf.fit(dtrain, dtarget, eval_metric="auc", eval_set=[(dtrain_test, dtrain_test_target)], early_stopping_rounds=30)

category_clf = LogisticRegression(dual = False, penalty='l2', random_state=13, multi_class='ovr', verbose=1, 
                                    n_jobs=4, solver='sag', C=3.9516364538326187, tol=0.060459326089611576)
category_clf.fit(dcat_train, dcat_target)

def objective(params):
    xgb_pred = xgb_clf.predict_proba(dtest)[:,1]
    log_pred = category_clf.predict_proba(dcat_test)[:,1]
    
    pred = xgb_pred + params['cat_k'] * log_pred
    for i in range(len(pred)):
        if pred[i] >= params['threshold']:
            pred[i] = 1
        else:
            pred[i] = 0
    
    auc = roc_auc_score(dtest_target, pred)
    print "auc {} params {}".format(auc, params)
    return {'loss':1-auc, 'status': STATUS_OK }
    
params = {
    'cat_k': hp.uniform('cat_k', 0.1, 1),
    'threshold': hp.uniform('threshold', 0, 1),
}    
iteration_number = 1000

trials = Trials()
best_res = fmin(fn=objective,
                space=params,
                algo=tpe.suggest,
                max_evals=iteration_number,
                trials=trials)
print "BEST "
print best_res

Will train until validation_0 error hasn't decreased in 30 rounds.
[0]	validation_0-auc:0.500000
[1]	validation_0-auc:0.500000
[2]	validation_0-auc:0.500000
[3]	validation_0-auc:0.500000
[4]	validation_0-auc:0.500000
[5]	validation_0-auc:0.704912
[6]	validation_0-auc:0.704912
[7]	validation_0-auc:0.758977
[8]	validation_0-auc:0.753610
[9]	validation_0-auc:0.753880
[10]	validation_0-auc:0.756737
[11]	validation_0-auc:0.756737
[12]	validation_0-auc:0.756605
[13]	validation_0-auc:0.756602
[14]	validation_0-auc:0.753987
[15]	validation_0-auc:0.758389
[16]	validation_0-auc:0.760716
[17]	validation_0-auc:0.764817
[18]	validation_0-auc:0.764835
[19]	validation_0-auc:0.764831
[20]	validation_0-auc:0.766268
[21]	validation_0-auc:0.767419
[22]	validation_0-auc:0.767325
[23]	validation_0-auc:0.767525
[24]	validation_0-auc:0.767430
[25]	validation_0-auc:0.769579
[26]	validation_0-auc:0.769537
[27]	validation_0-auc:0.770513
[28]	validation_0-auc:0.770305
[29]	validation_0-auc:0.770283
[30]	validati

convergence after 10 epochs took 2 seconds


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    2.0s finished


auc 0.5 params {'threshold': 0.6964691855978616, 'cat_k': 0.3575254014553415}
auc 0.706431116471 params {'threshold': 0.10606490595473272, 'cat_k': 0.7709243363970517}
auc 0.5 params {'threshold': 0.50672601149957, 'cat_k': 0.15177762565128078}
auc 0.53113177113 params {'threshold': 0.10517212721767522, 'cat_k': 0.21532966483060034}
auc 0.5 params {'threshold': 0.5238333199546759, 'cat_k': 0.13596667673497678}
auc 0.5 params {'threshold': 0.8666370428503979, 'cat_k': 0.3368303646376725}
auc 0.5 params {'threshold': 0.5705316251454068, 'cat_k': 0.34143379823251085}
auc 0.523337714085 params {'threshold': 0.1416302690767407, 'cat_k': 0.38038992501483104}
auc 0.500525762355 params {'threshold': 0.650153605471749, 'cat_k': 0.9532982492562428}
auc 0.5 params {'threshold': 0.7809038987924661, 'cat_k': 0.44303008763740737}
auc 0.5 params {'threshold': 0.4195086163487056, 'cat_k': 0.21669477335629372}
auc 0.5 params {'threshold': 0.8114036263085849, 'cat_k': 0.5029878203624306}
auc 0.5 params 

In [107]:
# best params 
auc 0.768332723492 params {'threshold': 0.01958767611980896, 'cat_k': 0.10053511401918697}

In [ ]:
# try another objective function
def objective(params):
    xgb_pred = xgb_clf.predict_proba(dtest)[:,1]
    log_pred = category_clf.predict_proba(dcat_test)[:,1]
    
    pred = params['xgb_k'] * xgb_pred**params['xgb_n'] + params['cat_k'] * log_pred**params['cat_n']
    for i in range(len(pred)):
        if pred[i] >= params['threshold']:
            pred[i] = 1
        else:
            pred[i] = 0
    
    auc = roc_auc_score(dtest_target, pred)
    print "auc {} params {}".format(auc, params)
    return {'loss':1-auc, 'status': STATUS_OK }
    
params = {
    'cat_k': hp.uniform('cat_k', 0, 1),
    'cat_n': hp.uniform('cat_n', 0, 2),
    'xgb_k': hp.uniform('xgb_k', 0, 1),
    'xgb_n': hp.uniform('xgb_n', 0, 2),
    'threshold': hp.uniform('threshold', 0, 10),
}    
iteration_number = 10000

trials = Trials()
best_res = fmin(fn=objective,
                space=params,
                algo=tpe.suggest,
                max_evals=iteration_number,
                trials=trials)
print "BEST "
print best_res


auc 0.5 params {'threshold': 2.268514535642031, 'xgb_k': 0.28613933495037946, 'cat_n': 1.1026295381657825, 'xgb_n': 1.3929383711957233, 'cat_k': 0.7194689697855631}
auc 0.5 params {'threshold': 5.723135366314988, 'xgb_k': 0.745471484885613, 'cat_n': 0.9164823675217628, 'xgb_n': 0.21212981190946545, 'cat_k': 0.3847059030441362}
auc 0.5 params {'threshold': 6.275799857910759, 'xgb_k': 0.05753069516808973, 'cat_n': 0.26510505440031973, 'xgb_n': 1.01345202299914, 'cat_k': 0.13108514971070295}
auc 0.5 params {'threshold': 0.8740630845952124, 'xgb_k': 0.12814407203400036, 'cat_n': 0.23095712534208368, 'xgb_n': 0.21034425443535043, 'cat_k': 0.7477276197510565}
auc 0.5 params {'threshold': 1.8596531220043622, 'xgb_k': 0.039962974149974184, 'cat_n': 1.545581295905817, 'xgb_n': 1.0476666399093517, 'cat_k': 0.5521488787070138}
auc 0.5 params {'threshold': 1.31408478173487, 'xgb_k': 0.2631448495974139, 'cat_n': 0.08318688749537473, 'xgb_n': 1.7332740857007958, 'cat_k': 0.23892433469051455}
auc 0.5

In [ ]:
# best parametrs 
auc 0.767830705243 {'threshold': 0.010177010937287788, 'xgb_k': 0.17652353342254384, 'cat_n': 2.2238509460958156, 'xgb_n': 0.7107432826000818, 'cat_k': 0.7078955901408212}

In [60]:
# best results are just using xgboost
xgb_clf = xgb.XGBClassifier(missing=np.nan, nthread=4, seed=13, 
                            scale_pos_weight = 0.3999090923487273,
                            learning_rate = 0.2674798971406099,
                            max_delta_step = 8.0,
                            base_score = 0.6868410504015201,
                            n_estimators = 34952,
                            subsample = 0.8866035180913567,
                            min_child_weight = 4.0,
                            max_depth = 4.0,
                            colsample_bytree=0.7)

xgb_clf.fit(train, target)
test_pred = xgb_clf.predict_proba(test)[:,1]

print test_pred

KeyboardInterrupt: 

In [105]:
submission = pd.DataFrame({"ID": test.index, "TARGET": test_pred})
submission.to_csv("/Users/Kseniya/study/4_term/ml/my_submission.csv", index=False)

In [ ]:
%%time
# previous testing. not importent now
from sklearn import svm

classes = [
  {'type': "svm linear", 'classifier': svm.SVC, 'params': {'kernel': 'linear' }},
  {'type': "svm poly 3", 'classifier': svm.SVC, 'params': {'kernel': 'poly', 'degree': '3' }},
  {'type': "svm poly 2", 'classifier': svm.SVC, 'params': {'kernel': 'poly', 'degree': '2' }}, 
  {'type': "svm linear", 'classifier': svm.SVC, 'params': {'kernel': 'sigmoid', 'coef': '1' }}, 
]

train_el = 500
for c in classes:
    params = dict(c.get('params', {}))
    classifier = c['classifier']
    classifier = classifier(**params)
    res = cross_validation.cross_val_score(classifier, train[:train_el], target[:train_el], n_jobs=4, scoring='roc_auc').mean()
    print "{}: {}\n".format(c['type'], res)

from sklearn.ensemble import AdaBoostClassifier
classes = [
    {'type': "ada boost 15", 'classifier': AdaBoostClassifier, 'params': {'n_estimators': 15}},
    {'type': "ada boost 10", 'classifier': AdaBoostClassifier, 'params': {'n_estimators': 10}},
]

for c in classes:
    params = dict(c.get('params', {}))
    classifier = c['classifier']
    classifier = classifier(**params)
    res = cross_validation.cross_val_score(classifier, train[:train_el], target[:train_el], n_jobs=4, scoring='roc_auc').mean()
    print "{}: {}\n".format(c['type'], res)
    